In [83]:
import pandas as pd
import math

In [84]:
equipment_dimension_lookup = {
    "MEETING ROOM CHAIRS": 0,
    "TABLES": 0
}

equipment_capacity_lookup = {
    "MEETING ROOM CHAIRS": 10.0,
    "TABLES": 15.0,
    
    "WORKER": 1
}

In [85]:
class Satchel:
    def __init__(self):
        self.equip_dictionary = {}
    def add_item(self, item, quantity):
        if item in self.equip_dictionary.keys():
            self.equip_dictionary[item] += quantity
        else:
            self.equip_dictionary[item] = quantity
    def remove_item(self, item, quantity):
        if item in self.equip_dictionary.keys():
            if self.equip_dictionary[item] < quantity:
                quantity = self.equip_dictionary[item]
                self.equip_dictionary[item] = 0
                return quantity
            else:
                self.equip_dictionary[item] -= quantity
                return quantity
        else:
            return 0
    def get_level(self, item):
        if item in self.equip_dictionary.keys():
            return self.equip_dictionary[item]
        else:
            return 0
    # get shortfall when compared to satchel_b
    def get_shortfall(self, satchel_b):
        shortfall = Satchel()
        for item in satchel_b.keys():
            cur_level = self.get_level(item)
            foreign_level = satchel_b.get_level(item)
            if (cur_level >= foreign_level):
                shortfall.add_item(item, 0)
            else:
                shortfall.add_item(item, foreign_level-cur_level)
        return shortfall
    def is_empty(self):
        for val in self.equip_dictionary.values():
            if val > 0:
                return False
        return True

In [112]:
class Room:
    def __init__(self, name, room_type, inventory, capacity, total_capacity):
        self.name = name
        self.inventory = inventory
        self.room_type = room_type
        self.distances = {}
        self.capacity  = capacity
        self.total_capacity = total_capacity
    def __init__(self, name, room_type, inventory):
        self.name = name
        self.inventory = inventory
        self.room_type = room_type
        self.distances = {}
        self.capacity  = Satchel()
        self.total_capacity = float('Inf')
    def __init__(self, name, room_type):
        self.name = name
        self.inventory = Satchel()
        self.room_type = room_type
        self.distances = {}
        self.capacity  = Satchel()
        self.total_capacity = float('Inf')
    def __hash__(self):
        return hash(self.name)
    def __eq__(self, other):
        return self.name == other.name
    def get_cumulative_dimension(self):
        cumulative_dimension = 0
        for item in self.inventory.equip_dictionary.keys():
            cumulative_dimension += self.inventory.get_level(item) * equipment_dimension_lookup[item]
        return cumulative_dimension
    def load_distances(self, distance_table, room_list):
        for room in room_list:
            if room == self:
                continue
            else:
                self.distances[room] = distance_table[(distance_table["start"] == self.name) & (distance_table["end"] == room.name)]["distance"].min()
    def get_closest_room (self, room_type):
        min_room = Room("dummy", "na")
        min_distance = float('Inf')
        for room in self.distances.keys():
            if self.distances[room] < min_distance and room.room_type == room_type:
                min_room = room
                min_distance = self.distances[room]
        return min_room
    def remove_item(self,item, quantity):
        return self.inventory.remove_item(item, quantity)
    def add_item(self, item, quantity):
        cumulative_shortfall = 0
        if self.get_cumulative_dimension() + quantity * equipment_dimension_lookup[item] > self.total_capacity:
            new_dimension = self.get_cumulative_dimension() - self.total_capacity
            if new_dimension < 0:
                return 0
            else:
                new_quantity = math.floor(new_dimension/equipment_dimension_lookup[item])
                cumulative_shortfall = quantity - new_quantity
                quantity = new_quantity
        if self.inventory.get_level(item) + quantity > self.capacity.get_level(item):
            amount_to_add = self.capacity.get_level(item) - self.inventory.get_level(item)
            self.inventory.add_item(item, amount_to_add)
            return quantity - amount_to_add + cumulative_shortfall
        else:
            self.inventory.add_item(item, quantity)
            return cumulative_shortfall

In [129]:
# requirements: a list of [(Event Room), Equipment, Quantity, Direction] representing requirements
#     Direction: True if equipment is coming in, False if equipment is coming out
def build_naive_movement(rooms, requirements):
    movement_matrix = pd.DataFrame(columns=["equip_type", "quantity", "start", "end"])
    prev_location = "null"
    for tup in requirements:
        event_room = tup[0]
        item = tup[1]
        quantity = tup[2]
        direction = tup[3]
        
        if prev_location == "null":
            prev_location = event_room.name
        else:
            movement_matrix = movement_matrix.append({"equip_type": "WORKER", "quantity": 1, "start": prev_location, "end": event_room.name}, ignore_index=True)
            prev_location = event_room.name
        
        if direction:v
            # go to closest storage room recursively until all quantity has been gathered
            storage_room = event_room.get_closest_room("storage")
            quantity_fulfilled = 0
            trip_quantity = 0
            while quantity_fulfilled < quantity:
                if quantity_fulfilled == 0:
                    movement_matrix = movement_matrix.append({"equip_type": "WORKER", "quantity": 1, "start": prev_location, "end": storage_room.name}, ignore_index = True)
                else:
                    movement_matrix = movement_matrix.append({"equip_type": item, "quantity": trip_quantity, "start": prev_location, "end": storage_room.name}, ignore_index = True)
                trip_quantity += storage_room.remove_item(item, quantity - quantity_fulfilled)
                while trip_quantity > equipment_capacity_lookup[item]:
                    movement_matrix = movement_matrix.append({"equip_type": item, "quantity": equipment_capacity_lookup[item], "start": storage_room.name, "end": event_room.name}, ignore_index = True)
                    movement_matrix = movement_matrix.append({"equip_type": "WORKER", "quantity": 1, "start": event_room.name, "end": storage_room.name}, ignore_index = True)
                    trip_quantity -= equipment_capacity_lookup[item]
                    quantity_fulfilled += equipment_capacity_lookup[item]
                prev_location = storage_room.name
                storage_room = storage_room.get_closest_room("storage")
                quantity_fulfilled += trip_quantity
            # return to event room
            movement_matrix = movement_matrix.append({"equip_type": item, "quantity": trip_quantity, "start": prev_location, "end": event_room.name}, ignore_index = True)
        else:
            # go to closest storage room recursively
            storage_room = event_room.get_closest_room("storage")
            quantity_to_dispose = quantity
            trip_quantity = 0
            while quantity_to_dispose > 0:
                trip_quantity = min(equipment_capacity_lookup[item], quantity_to_dispose)
                quantity_to_dispose -= trip_quantity
                while trip_quantity > 0:
                    movement_matrix = movement_matrix.append({"equip_type": item, "quantity": trip_quantity, "start": prev_location, "end": storage_room.name}, ignore_index = True)
                    trip_quantity = storage_room.add_item(item, trip_quantity)
                    if trip_quantity > 0:
                        prev_location = storage_room.name
                        storage_room = storage_room.get_closest_room("storage")
                movement_matrix = movement_matrix.append({"equip_type": "WORKER", "quantity": 1, "start": storage_room.name, "end": event_room.name}, ignore_index = True)
    
    return movement_matrix

In [141]:
def enhanced_cost_function(movement_matrix, distance_matrix, equip_capacity, speed):
    movement_matrix = movement_matrix.merge(distance_matrix, on=["start", "end"])
    movement_matrix["trips"] = movement_matrix.apply(lambda x: math.ceil(x["quantity"]/equip_capacity[x["equip_type"]]), axis=1)
    movement_matrix["cost"] = movement_matrix["trips"] * movement_matrix["distance"] / speed
    return movement_matrix["cost"].sum()

In [89]:
movement_list = {
                "equip_type": ["MEETING ROOM CHAIRS", "MEETING ROOM CHAIRS", "TABLES"],
                 "quantity":   [30, 45, 17],
                 "start":      ["A", "B", "C"],
                 "end":        ["B", "A", "D"]
                }
distance_list = {
                "start":    ["A", "A", "A", "B", "B", "B", "C", "C", "C", "D", "D", "D"],
                "end":      ["B", "C", "D", "A", "C", "D", "A", "B", "D", "A", "B", "C"],
                "distance": [100, 200, 300, 100, 250, 275, 200, 250, 50, 300, 275, 50]
}

In [90]:
movements = pd.DataFrame(movement_list)
distances = pd.DataFrame(distance_list)

In [91]:
movements

,equip_type,quantity,start,end
0,MEETING ROOM CHAIRS,30,A,B
1,MEETING ROOM CHAIRS,45,B,A
2,TABLES,17,C,D


In [92]:
enhanced_cost_function(movements, distances, equipment_capacity_lookup, 5)

180.0

In [93]:
distances

,start,end,distance
0,A,B,100
1,A,C,200
2,A,D,300
3,B,A,100
4,B,C,250
5,B,D,275
6,C,A,200
7,C,B,250
8,C,D,50
9,D,A,300


In [130]:
rooms = [Room("A", "event"), Room("B", "storage"), Room("C", "storage"), Room("D", "event")]

In [131]:
for room in rooms:
    room.load_distances(distances, rooms)

In [132]:
rooms[0].distances

{<__main__.Room at 0x7ff862184e48>: 100,
 <__main__.Room at 0x7ff8621849b0>: 200,
 <__main__.Room at 0x7ff8621428d0>: 300}

In [133]:
rooms[1].capacity.add_item("MEETING ROOM CHAIRS", 200)
rooms[1].capacity.add_item("TABLES", 200)
rooms[2].capacity.add_item("MEETING ROOM CHAIRS", 150)
rooms[2].capacity.add_item("TABLES", 50)
rooms[1].inventory.add_item("MEETING ROOM CHAIRS", 25)
rooms[1].inventory.add_item("TABLES", 5)
rooms[2].inventory.add_item("MEETING ROOM CHAIRS", 100)
rooms[2].inventory.add_item("TABLES", 45)

rooms[0].capacity.add_item("MEETING ROOM CHAIRS", 1000)
rooms[0].capacity.add_item("TABLES", 1000)
rooms[3].capacity.add_item("MEETING ROOM CHAIRS", 1000)
rooms[3].capacity.add_item("TABLES", 1000)

In [134]:
reqs = [[rooms[0], "MEETING ROOM CHAIRS", 45, True],
        [rooms[3], "MEETING ROOM CHAIRS", 12, True],
        [rooms[0], "TABLES", 18, True],
        [rooms[0], "MEETING ROOM CHAIRS", 15, False]]

In [135]:
m_matrix = build_naive_movement(rooms, reqs)

In [136]:
m_matrix

,equip_type,quantity,start,end
0,WORKER,1,A,B
1,MEETING ROOM CHAIRS,10,B,A
2,WORKER,1,A,B
3,MEETING ROOM CHAIRS,10,B,A
4,WORKER,1,A,B
5,MEETING ROOM CHAIRS,5,B,C
6,MEETING ROOM CHAIRS,10,C,A
7,WORKER,1,A,C
8,MEETING ROOM CHAIRS,10,C,A
9,WORKER,1,A,C


In [140]:
enhanced_cost_function(m_matrix, distances, equipment_capacity_lookup, 5)

             equip_type quantity start end  distance  trips  cost
0                WORKER        1     A   B       100      1  20.0
1                WORKER        1     A   B       100      1  20.0
2                WORKER        1     A   B       100      1  20.0
3                WORKER        1     A   B       100      1  20.0
4   MEETING ROOM CHAIRS       10     A   B       100      1  20.0
5   MEETING ROOM CHAIRS        5     A   B       100      1  20.0
6   MEETING ROOM CHAIRS       10     B   A       100      1  20.0
7   MEETING ROOM CHAIRS       10     B   A       100      1  20.0
8                WORKER        1     B   A       100      1  20.0
9                WORKER        1     B   A       100      1  20.0
10  MEETING ROOM CHAIRS        5     B   C       250      1  50.0
11               TABLES        5     B   C       250      1  50.0
12  MEETING ROOM CHAIRS       10     C   A       200      1  40.0
13  MEETING ROOM CHAIRS       10     C   A       200      1  40.0
14  MEETIN

750.0

In [138]:
distances

,start,end,distance
0,A,B,100
1,A,C,200
2,A,D,300
3,B,A,100
4,B,C,250
5,B,D,275
6,C,A,200
7,C,B,250
8,C,D,50
9,D,A,300
